# Regresja liniowa za pomocą TensorFlow

In [ ]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## Przygotowanie danych

Ponownie wykorzystamy problem przewidywania cen domów w Bostonie. Poniższy kod wczytuje dane i dzieli na trzy podzbiory: uczący, walidujący i testowy w proporcji 70%/10%/20%

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
X = boston['data']
y = boston['target']

In [ ]:
n, p = X.shape
n_train = int(.7*n)
n_validation = int(.1*n)
indices = np.random.permutation(n)
X = X[indices]
y = y[indices]
X_train = X[:n_train, :]
y_train = y[:n_train]
X_validation = X[n_train:n_train+n_validation, :]
y_validation = y[n_train:n_train+n_validation]
X_test = X[n_train+n_validation:, :]
y_test = y[n_train+n_validation:]

## Konstrukcja grafu obliczeń

TensorFlow oparty jest na koncepcji grafu obliczeń. Raz skonstruowany graf można wykorzystywać dla różnych danych wejściowych. Taka architektura umożliwa też automatyczne obliczanie gradientów i ich propagację w procesie optymalizacji. Konstrukcję grafu rozpoczniemy od usunięcia wszystkich operacji z domyślnego grafu.

In [ ]:
tf.reset_default_graph()

Utworzymy placeholdery na dane wejściowe: macierz X i macierz y. Placeholdery można traktować jako argumenty funkcji: dopóki funkcja nie zostanie wywołana, to zmienne nie mają wartości - tak samo placeholdery nie mają wartości, dopóki nie uruchomimy obliczeń w grafie. Funkcja `tf.placeholder` wymaga dwóch argumentów: typu danych (tutaj: `tf.float32`, czyli 32-bitowy typ zmiennoprzecinkowy) oraz typu (shape) tensora (macierzy, wektora itd.), który zostanie podany. Opcjonalnie pierwszy wymiar może zostać zastąpiony przez `None` i wtedy można podawać tensory o różnych długościach w pierwszym wymiarze, natomiast pozostałe wymiary muszą być zdefiniowane.

In [ ]:
X_pl = tf.placeholder(tf.float32, shape=(None, p)) # Macierz 2D, dowolna liczba wierszy, p kolumn
y_pl = tf.placeholder(tf.float32, shape=(None,)) # Wektor o dowolnej liczbie wierszy

Następnie utworzymy w grafie zmienne: zmienną `W` w formie wektora typu $p\times 1$ oraz zmienną skalarną `b`. Obie te zmienne będą podlegały optymalizacji w procesie uczenia.

In [ ]:
W = tf.get_variable("W", dtype=tf.float32, shape=(p,1))
b = tf.get_variable("b", dtype=tf.float32, shape=(1,))

Dodajemy do grafu dwie operacje: obliczania wartość predykcji (wektor `y_pred`) oraz obliczania wartość błędu średniokwadratowego.

In [ ]:
y_pred = X_pl@W + b
mse = tf.reduce_mean((y_pred-y_pl)**2)

Chcemy, żeby zmienne `W` oraz `b` automatycznie się zoptyamlizowały w procesie uczenia. Wykorzystamy w tym celu klasę `tf.train.AdamOptimizer`, która implementuje pewne ulepszenie algorytmu Gradient Descent. Dodajemy do grafu operator `minimizer`, wygenerowany przez instancję klasy `AdamOptimizer`. Celem tego operatora jest minimalizowane wartości `mse` podanej jako argument.

In [ ]:
opt = tf.train.AdamOptimizer()
minimizer = opt.minimize(mse)

Utworzone w procesie konstrukcji grafu zmienne nie mają przypisanych wartości początkowych. Żeby to zrobić dodajemy do grafu kolejny operator.

In [ ]:
init = tf.global_variables_initializer()

## Uruchomienie obliczeń

Wykonywanie obliczeń zdefiniowanych przez graf odbywa się za pomocą sesji. Rozpoczniemy od utworzenia obiektu sesji i wykonania operatora `init`, który przypisze wartości początkowe zmiennym.

In [ ]:
sess = tf.InteractiveSession()
sess.run(init)

Zaimplementujemy uczenie mini-batch przez `n_epoch` epok. 

1. Rozpoczynamy od wybrania `batch_size` indeksów spośród wszystkich `n_train` obiektów uczących. 
2. Przygotowujemy słownik `feed_dict`, w którym kluczami są placeholdery, a wartościami wartości, które mają zostać przypisane placeholderom. 
3. Każemy obliczyć wartości wyrażeń `minimizer` i `mse`, podając `feed_dict` jako słownik z wartościami placeholderów. TensorFlow wykonuje obliczenia inteligentnie w tym sensie, że do wykonania minimizer musi obliczyć ileś wyrażeń pośrednich, w tym wartość wyrażenia `mse`: zostaną wykonane wyłącznie niezbędne działania w grafie i każde dokładnie raz. Wartość wyrażenia `mse` zostanie obliczona również tylko raz, ale ponieważ jawnie podaliśmy ją do wykonania, jej wartość zostanie zwrócona jako wynik wykonania metody `run`.
4. Zapamiętujemy wartość `mse`

In [ ]:
mse_values = []
batch_size = 100
n_epoch = 1000
for epoch in range(n_epoch):
    indices = np.random.choice(n_train, size=batch_size)
    feed_dict = {X_pl: X_train[indices,:], y_pl: y_train[indices]}
    _, mse_value = sess.run([minimizer, mse], feed_dict)
    mse_values.append(mse_value)
plt.plot(mse_values)
plt.show()

Uruchom ponownie powyższą komórkę z kodem. Zaobserwuj, że wykres wygląda  zupełnie inaczej: sesja jest ciągle aktywna, więc wartości zmiennych są pamiętane. Ponieważ zmienne `W` i `b` zostały już częściowo zoptymalizowane, wiec i wykres wygląda zupełnie inaczej. Żeby powrócić do stanu początkowego musisz ponownie wywołać operator `init`.

## Early stopping

Zaimplementuj early stopping: Co 100-tną epokę uczenia oblicz wartość `mse` podając jako wartość `X_pl` macierz `X_validation`, a jako `y_pl` y_validation. Pamiętaj, żeby wtedy **nie** uruchamiać operatora `minimizer`, bo na zbiorze walidującym nie dokonujemy uczenia. Zapamiętuj najlepsze wartości `mse` i numery epok uczenia, w których je osiągnięto. Przerwij uczenie jeżeli przez ostatnie 1000 epok nie udało się znaleźć lepszej wartości `mse`. Zapamiętuj warotści `mse` uzyskane podczas uczenia i podczas walidacji (odpowiednio w `train_mses` i `validation_mses`), a następnie narysuj je na wykresie. Wypisz, w którym kroku zostało przerwane uczenie.

In [ ]:
sess.run(init)
train_mses = []
validation_mses = []
for epoch in range(50000):
    ...
plt.plot(...)
plt.plot(...)
plt.show()

## Regularyzacja L1

Zaimplementuj regularyzację L1:

1. Utwórz placeholder na hiperparameter alpha
2. Utwórz operator `cost` zgodnie z poniższym wzorem: 
$$ cost = MSE + \alpha \sum_{i=1}^n \left|w_i\right| $$
Do zsumowania wszystkich wartości z macierzy `W` wykorzystaj funkcję `tf.reduce_sum`, a do obliczenia wartości bezwzględnej `tf.abs`.
3. Utwórz nowy operator do optymalizacji wartości `cost` i nowy operator do inicjowania wartości zmiennych `init`

In [ ]:
alpha_pl = tf.placeholder(tf.float32)
cost = mse + alpha_pl*tf.reduce_sum(tf.abs(W))
cost_minimizer = tf.train.AdamOptimizer().minimize(cost)
init = tf.global_variables_initializer()

Przetestuj uzyskane rozwiązanie: ucz przez 10000 epok dla wartości każdej z następujacych wartości $\alpha$: 0.01, 0.1, 1, 10, 100. Za każdym razem uruchamiaj operator `init`. Zbieraj co 100 epok MSE na zbiorze walidującym i narysuj je na wspólnym wykresie dla wszystkich 5 wartości hiperparametru $\alpha$. Która z wartości $\alpha$ jest najlepsza?

In [ ]:
mses = {}
for alpha in [0.01, 0.1, 1, 10, 100]:    
    sess.run(init)
    mses[alpha] = [] # tu zahcowuj wartości błędu na zbiorze walidującym
    for epoch in range(10000):
        ...
        if epoch % 100 == 0:
            ...

In [ ]:
for alpha in sorted(mses.keys()):
    plt.plot(mses[alpha])
plt.legend([str(alpha) for alpha in sorted(mses.keys())])
plt.show()
for alpha in sorted(mses.keys()):
    plt.plot(mses[alpha][-10:])
plt.legend([str(alpha) for alpha in sorted(mses.keys())])
plt.show()

Która z wartości $\alpha$ jest najlepsza? ...